# Grid5k cluster installation

In [3]:
%run ../common/common.ipynb

Note: you may need to restart the kernel to use updated packages.


## Prerequisites

Launch this notebook using a Grid5k's Jupyter notebook on the frontend (https://intranet.grid5000.fr/notebooks). Here we do everything in Nancy.


- install Terraform 0.14.11
- usage Terraform Grid5000 K8S (Rancher) provider https://github.com/pmorillon/terraform-grid5000-k8s-cluster using:
    - Terraform Grid5000 provider https://registry.terraform.io/providers/pmorillon/grid5000/0.0.7
    - Rancher rke / https://registry.terraform.io/providers/rancher/rke/1.2.1

- terraform plan
- terraform untaint module.k8s_cluster.grid5000_job.k8s
- terraform apply

## Cluster configuration

Set `main.tf` with wanted values (number of nodes, walltime etc.).

In [4]:
%%bash
export PATH=$HOME/tools:$PATH
terraform workspace select default 
rm terraform.tfstate # remove previous state
terraform init
terraform apply -auto-approve

rm: cannot remove 'terraform.tfstate': No such file or directory


Initializing modules...

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of pmorillon/grid5000 from the dependency lock file
- Reusing previous version of rancher/rke from the dependency lock file
- Reusing previous version of hashicorp/null from the dependency lock file
- Reusing previous version of hashicorp/local from the dependency lock file
- Using previously-installed pmorillon/grid5000 v0.0.10
- Using previously-installed rancher/rke v1.3.0
- Using previously-installed hashicorp/null v3.2.1
- Using previously-installed hashicorp/local v2.4.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and r

In [5]:
%%bash
mkdir -p $HOME/.kube
cp kube_config_cluster.yml $HOME/.kube/config
export PATH=$HOME/tools:$PATH
#export KUBECONFIG=$PWD/kube_config_cluster.yml
oarstat -u 
# use oardel to remove the instances
kubectl version
kubectl get nodes

Job id     Name           User           Submission Date     S Queue
---------- -------------- -------------- ------------------- - ----------
4211373    gepiciad_resou grosinosky     2023-06-27 11:51:47 R default   
Client Version: version.Info{Major:"1", Minor:"22", GitVersion:"v1.22.0", GitCommit:"c2b5237ccd9c0f1d600d3072634ca66cefdf272f", GitTreeState:"clean", BuildDate:"2021-08-04T18:03:20Z", GoVersion:"go1.16.6", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"22", GitVersion:"v1.22.4", GitCommit:"b695d79d4f967c403a96986f1750a35eb75e75f1", GitTreeState:"clean", BuildDate:"2021-11-17T15:42:41Z", GoVersion:"go1.16.10", Compiler:"gc", Platform:"linux/amd64"}
NAME                         STATUS     ROLES               AGE   VERSION
gros-1.nancy.grid5000.fr     NotReady   controlplane,etcd   46s   v1.22.4
gros-10.nancy.grid5000.fr    NotReady   worker              44s   v1.22.4
gros-102.nancy.grid5000.fr   NotReady   worker              45s   v1.2

Nodes are reachable using ssh root@<machine>
```bash
 curl -X POST 'https://api.grid5000.fr/3.0/sites/nancy/storage/home/grosinosky/access' -H "Content-Type: application/json" -d '{"termination" : {"job": 3333824, "site": "nancy"}}'
```
This command adds the capacity to mount the g5k $HOME directory. It should be done manually if necessary.


## Label nodes

In [6]:
init_label_nodes(jobmanagers_qty=2, kafka_qty=0)
#init_label_nodes(jobmanagers_qty=8, kafka_qty=16)

gros-1.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-1.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/controlplane': 'true', 'node-role.kubernetes.io/etcd': 'true'}
gros-10.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-10.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/worker': 'true'}
gros-102.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-102.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/worker': 'true'}
gros-103.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-103.nancy.grid5000.fr', 'kubern

gros-45.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-45.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/worker': 'true'}
gros-54.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-54.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/worker': 'true'}
gros-55.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-55.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/worker': 'true'}
gros-56.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-56.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/w

In [7]:
(manager_node, jobmanager_node, taskmanager_nodes) = get_label_nodes()

gros-1.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-1.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/controlplane': 'true', 'node-role.kubernetes.io/etcd': 'true'}
gros-10.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-10.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/worker': 'true', 'tier': 'manager'}
gros-102.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostname': 'gros-102.nancy.grid5000.fr', 'kubernetes.io/os': 'linux', 'node-role.kubernetes.io/worker': 'true', 'tier': 'jobmanager'}
gros-103.nancy.grid5000.fr	{'beta.kubernetes.io/arch': 'amd64', 'beta.kubernetes.io/os': 'linux', 'kubernetes.io/arch': 'amd64', 'kubernetes.io/hostnam

In [8]:
run_command('../common/common_modules.sh')

../common/common_modules.sh
../common/common_modules.sh


Log 0 - 2023-06-27 12:10:30 : error: unable to recognize "./cluster-role-binding-default.yaml": no matches for kind "ClusterRoleBinding" in version "rbac.authorization.k8s.io/v1beta1"
Log 0 - 2023-06-27 12:10:32 : customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io created
Log 0 - 2023-06-27 12:10:32 : customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io created
Log 0 - 2023-06-27 12:10:32 : customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io created
Log 0 - 2023-06-27 12:10:32 : customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io created
Log 0 - 2023-06-27 12:10:32 : customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io created
Log 0 - 2023-06-27 12:10:32 : customresourcedefinition.apiextensions.k8s.io/orders.acme.cert-manager.io created
Log 0 - 2023-06-27 12:10:32 : namespace/cert-manager created
Log 0 - 2023-06-27 12:10:32 : serviceaccount/cert-manager-cainjector cr

0

In [9]:
print("Access to Minio: http://{}:30900".format(manager_node))
print("Access to Grafana: http://{}:30300".format(manager_node))
print("Access to Prometheus: http://{}:30090".format(manager_node))
print("Job manager address: {}".format(jobmanager_node))
print("Task manager addresses: {}".format(taskmanager_nodes))

Access to Minio: http://gros-10.nancy.grid5000.fr:30900
Access to Grafana: http://gros-10.nancy.grid5000.fr:30300
Access to Prometheus: http://gros-10.nancy.grid5000.fr:30090
Job manager address: gros-103.nancy.grid5000.fr
Task manager addresses: ['gros-105.nancy.grid5000.fr', 'gros-108.nancy.grid5000.fr', 'gros-116.nancy.grid5000.fr', 'gros-119.nancy.grid5000.fr', 'gros-22.nancy.grid5000.fr', 'gros-45.nancy.grid5000.fr', 'gros-54.nancy.grid5000.fr', 'gros-55.nancy.grid5000.fr', 'gros-56.nancy.grid5000.fr', 'gros-57.nancy.grid5000.fr', 'gros-58.nancy.grid5000.fr', 'gros-59.nancy.grid5000.fr', 'gros-6.nancy.grid5000.fr', 'gros-60.nancy.grid5000.fr', 'gros-61.nancy.grid5000.fr', 'gros-63.nancy.grid5000.fr', 'gros-64.nancy.grid5000.fr', 'gros-65.nancy.grid5000.fr', 'gros-66.nancy.grid5000.fr', 'gros-67.nancy.grid5000.fr', 'gros-68.nancy.grid5000.fr', 'gros-69.nancy.grid5000.fr', 'gros-7.nancy.grid5000.fr', 'gros-70.nancy.grid5000.fr', 'gros-71.nancy.grid5000.fr', 'gros-72.nancy.grid5000.f

In [11]:
%%bash
pwd
export PATH=$PATH:~/tools
export ZEPPELIN_POD=`kubectl get pods -n manager -o=name | grep zeppelin | sed "s/^.\{4\}//"` 
kubectl wait -n manager pod/$ZEPPELIN_POD --for condition=Ready --timeout=600s
export MANAGER_NODE=`kubectl get node --show-labels |grep tier=manager | awk '{print $1}'`

kubectl -n manager exec $ZEPPELIN_POD -- wget -P /tmp https://repo.maven.apache.org/maven2/org/apache/flink/flink-sql-connector-kafka_2.12/1.14.2/flink-sql-connector-kafka_2.12-1.14.2.jar
# nexmark and the rate limit connector should be both in the tmp directory
kubectl cp -n manager ../../tmp/flink-sql-connector-kafka-ratelimit_2.12-1.14.2.jar $ZEPPELIN_POD:/tmp/flink-sql-connector-kafka-ratelimit_2.12-1.14.2.jar
kubectl cp -n manager ../../tmp/nexmark-flink-0.2-SNAPSHOT.jar $ZEPPELIN_POD:/tmp/nexmark-flink-0.2-SNAPSHOT.jar
# upload zeppelin notebooks
for FILE in ../../xp/streambed-nexmark/zeppelin/*.zpln; do echo $FILE; curl ${MANAGER_NODE}:30088/api/notebook/import -d @$FILE; done
# set Flink interpreter to isolated mode (with default parameters)
curl -X PUT  ${MANAGER_NODE}:30088/api/interpreter/setting/flink -H 'Content-Type: application/json' -d @../common/zeppelin-flink-config.json
# remove from 
kubectl -n manager exec $ZEPPELIN_POD  -- rm /opt/flink/lib/flink-connector-kafka_2.12-1.14.2.jar
kubectl -n manager exec $ZEPPELIN_POD  -- rm /opt/flink/lib/flink-sql-connector-kafka_2.12-1.14.2.jar
curl -u "admin:prom-operator" -X POST ${MANAGER_NODE}:30300/api/dashboards/import -H 'Content-Type: application/json' -d "{\"Dashboard\":$(cat ../common/grafana.json)}"

/home/grosinosky/xp/gepiciad/resource-estimator/infra/g5k


pod/zeppelin-server-75767d75dd-zrkmg condition met


Defaulted container "zeppelin-server" out of: zeppelin-server, zeppelin-server-gateway, dnsmasq, flink-downloader (init)
--2023-06-27 10:16:29--  https://repo.maven.apache.org/maven2/org/apache/flink/flink-sql-connector-kafka_2.12/1.14.2/flink-sql-connector-kafka_2.12-1.14.2.jar
Resolving repo.maven.apache.org (repo.maven.apache.org)... 146.75.72.215, 2a04:4e42:82::215
Connecting to repo.maven.apache.org (repo.maven.apache.org)|146.75.72.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703507 (3.5M) [application/java-archive]
Saving to: ‘/tmp/flink-sql-connector-kafka_2.12-1.14.2.jar.1’

     0K .......... .......... .......... .......... ..........  1% 1.91M 2s
    50K .......... .......... .......... .......... ..........  2% 2.54M 2s
   100K .......... .......... .......... .......... ..........  4% 11.2M 1s
   150K .......... .......... .......... .......... ..........  5% 3.48M 1s
   200K .......... .......... .......... .......... ..........  6% 19.6

../../xp/streambed-nexmark/zeppelin/xp_datagen_2HY61EX49.zpln


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20573  100  6306  100 14267   615k  1393k --:--:-- --:--:-- --:--:-- 2009k


...

{"exception":"IOException","message":"java.io.IOException: Fail to import note: Note \u0027/xp_intro_q5_kafka_custom_ratelimit\u0027 existed","stacktrace":"java.io.IOException: java.io.IOException: Fail to import note: Note \u0027/xp_intro_q5_kafka_custom_ratelimit\u0027 existed\n\tat org.apache.zeppelin.rest.AbstractRestApi$RestServiceCallback.onFailure(AbstractRestApi.java:54)\n\tat org.apache.zeppelin.service.NotebookService.importNote(NotebookService.java:312)\n\tat org.apache.zeppelin.rest.NotebookRestApi.importNote(NotebookRestApi.java:369)\n\tat sun.reflect.GeneratedMethodAccessor9.invoke(Unknown Source)\n\tat sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.lang.reflect.Method.invoke(Method.java:498)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationHandlerFactory.java:52)\n\tat org.glassfish.jersey.server.model.internal.AbstractJavaResourceMethodDispatc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22865  100  6818  100 16047  1664k  3917k --:--:-- --:--:-- --:--:-- 5582k


{"exception":"IOException","message":"java.io.IOException: Fail to import note: Note \u0027/xp_intro_q8_kafka\u0027 existed","stacktrace":"java.io.IOException: java.io.IOException: Fail to import note: Note \u0027/xp_intro_q8_kafka\u0027 existed\n\tat org.apache.zeppelin.rest.AbstractRestApi$RestServiceCallback.onFailure(AbstractRestApi.java:54)\n\tat org.apache.zeppelin.service.NotebookService.importNote(NotebookService.java:312)\n\tat org.apache.zeppelin.rest.NotebookRestApi.importNote(NotebookRestApi.java:369)\n\tat sun.reflect.GeneratedMethodAccessor9.invoke(Unknown Source)\n\tat sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.lang.reflect.Method.invoke(Method.java:498)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationHandlerFactory.java:52)\n\tat org.glassfish.jersey.server.model.internal.AbstractJavaResourceMethodDispatcher$1.run(AbstractJavaResourceMeth

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25780  100  6886  100 18894  1681k  4612k --:--:-- --:--:-- --:--:-- 6293k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13767  100  6523  100  7244  1061k  1179k --:--:-- --:--:-- --:--:-- 2240k
Defaulted container "zeppelin-server" out of: zeppelin-server, zeppelin-server-gateway, dnsmasq, flink-downloader (init)
rm: cannot remove '/opt/flink/lib/flink-connector-kafka_2.12-1.14.2.jar': No such file or directory
command terminated with exit code 1
Defaulted container "zeppelin-server" out of: zeppelin-server, zeppelin-server-gateway, dnsmasq, flink-downloader (init)
rm: cannot remove '/opt/flink/lib/flink-sql-connector-kafka_2.12-1.14.2.jar': No such file or directory
command terminated with exit code 1
  % Tota

{"exception":"IOException","message":"java.io.IOException: Fail to import note: Note \u0027/xp_intro_q8_kafka_custom_ratelimit\u0027 existed","stacktrace":"java.io.IOException: java.io.IOException: Fail to import note: Note \u0027/xp_intro_q8_kafka_custom_ratelimit\u0027 existed\n\tat org.apache.zeppelin.rest.AbstractRestApi$RestServiceCallback.onFailure(AbstractRestApi.java:54)\n\tat org.apache.zeppelin.service.NotebookService.importNote(NotebookService.java:312)\n\tat org.apache.zeppelin.rest.NotebookRestApi.importNote(NotebookRestApi.java:369)\n\tat sun.reflect.GeneratedMethodAccessor9.invoke(Unknown Source)\n\tat sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.lang.reflect.Method.invoke(Method.java:498)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationHandlerFactory.java:52)\n\tat org.glassfish.jersey.server.model.internal.AbstractJavaResourceMethodDispatc

Import [Flink dashboard](https://grafana.com/grafana/dashboards/14911) in Grafana.

## Strimzi Kafka operator installation 

Installation based on https://strimzi.io/quickstarts/ and https://strimzi.io/docs/operators/in-development/quickstart.html#proc-install-product-str
The operator and the Kafka clusters and dependencies will deploy in the kafka namespace. 
Caution: operator seems can take time to make reconciliation especially in case of non working deployment. Please check the pod operator logs if something is not working.

A Kafka cluster sample is available in tpcds directory.